In [7]:
import tensorflow
import numpy
import matplotlib.pyplot as plt
import mnist

In [16]:
train = mnist.MNIST().load_training()
test = mnist.MNIST.load_testing()

FileNotFoundError: [Errno 2] No such file or directory: './train-labels-idx1-ubyte'

In [10]:
data.display()

TypeError: display() missing 1 required positional argument: 'img'

In [5]:
def sigmoid(z):
    res = 1./(1.+np.exp(-z))
    return (res, z)

def relu(z):
    return (np.maximum(0, z), z)
    
    
def sigmoid_backward(dA, cache):
    Z = cache
    A = sigmoid(Z)[0]
    dx = dA*(A*(1-A))
    return dx
    

def relu_backward(dA, cache):
    z = cache
    dz = np.array(dA, copy=True)
    dz[z <= 0] = 0
    return dz


def initialize_parameters(layers_dims):
    np.random.seed(3)
    parameters = []
    for i in range(1,len(layers_dims)):
        layer_params = {}
        layer_params['W'] = np.random.randn(layers_dims[i], layers_dims[i-1])*0.01
        layer_params['b'] = np.zeros((layers_dims[i], 1))
        parameters.append(layer_params)
    return parameters


def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -np.sum(np.multiply(Y, np.log(AL))+np.multiply((1-Y),np.log(1-AL)))/m
    
    cost = np.squeeze(cost) 
    assert(cost.shape == ())
    
    return cost


def linear(cache):
    A, W, b = cache
    return np.dot(W, A) + b


def linear_forward(A, W, b, threads=1):
    n_n = W.shape[0]
    Z = None
    if threads > 1 and n_n > 1:
        threshold = int(n_n/threads)
        segments = []
        for i in range(threads):
            W_segment = W[i*threshold:(i+1)*threshold,:]
            b_segment = b[i*threshold:(i+1)*threshold,:]
            segments.append((A, W_segment, b_segment))
            
        segmentsZ = foreach(linear, segments, threads=threads, return_=True)
        Z = segmentsZ[0]
        for i in range(1, len(segmentsZ)):
            Z = np.append(Z, segmentsZ[i], axis=0)
    else:
        Z = np.dot(W, A) + b
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    return Z, cache


def linear_activation_forward(A_prev, W, b, activation, threads=1):
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)
    return A, cache


def forward_prop(parameters, X, threads=1):
    
    L = len(parameters)
    A = X
    caches = []
    
    for i in range(L-1):
        W = parameters[i]['W']
        b = parameters[i]['b']
        A, cache = linear_activation_forward(A, W, b, 'relu', threads)
        caches.append(cache)
        
    W = parameters[L-1]['W']
    b = parameters[L-1]['b']
    A, cache = linear_activation_forward(A, W, b, 'sigmoid', threads)
    caches.append(cache)
    
    return A, caches    


def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = np.dot(dZ, A_prev.T)/m
    db = np.sum(dZ, axis=1, keepdims=True)/m
    dA_prev = np.dot(W.T, dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db


def linear_activation_backward(dA, cache, activation):

    linear_cache, activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db


def backward_prop(dAL, caches):
    L = len(caches)
    dA = dAL
    grads = []
    dA, dW, db = linear_activation_backward(dA, caches[L-1], 'sigmoid')
    layer_grads = {'dW': dW, 'db': db}
    grads.append(layer_grads)
    for i in reversed(range(0, L-1)):
        dA, dW, db = linear_activation_backward(dA, caches[i], 'relu')
        layer_grads = {'dW': dW, 'db': db}
        grads.append(layer_grads)
    grads.reverse()
    return grads


def update_parameters(parameters, grads, learning_rate):
    for i in range(len(parameters)):
        parameters[i]['W'] = parameters[i]['W'] - grads[i]['dW']*learning_rate
        parameters[i]['b'] = parameters[i]['b'] - grads[i]['db']*learning_rate
    return parameters


def predict(parameters, X):
    probs = forward_prop(parameters=parameters, X=X)[0]
    return (probs > 0.5).astype(int)

 
def L_layer_model(X_train, Y_train, X_test, Y_test, layers_dims, num_iterations=2000, learning_rate=0.005, threads = 1):
    np.random.seed(1)
    layers_dims.insert(0, X_train.shape[0])
    parameters = initialize_parameters(layers_dims=layers_dims)
    costs = []
    iterations = []
    for i in range(num_iterations):
        AL, caches = forward_prop(parameters=parameters, X=X_train, threads=threads)
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(AL=AL, Y=Y_train)
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        dAL = -(np.divide(Y_train, AL) - np.divide(1 - Y_train, 1 - AL))
        grads = backward_prop(dAL, caches)
        
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(grads=grads, learning_rate=learning_rate, parameters=parameters)
        # Print the cost every 100 iterations
        if i % 100 == 0:
            iterations.append(i)
            costs.append(cost)
            print("Cost after iteration %i: %f" %(i, cost))
            
    Y_prediction_test = predict(parameters,X=X_test)
    Y_prediction_train = predict(parameters,X=X_train)
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    plt.plot(iterations, costs)
    plt.show()
    
    return parameters